# Evaluation Metrics

### Import Libraries and Dependencies

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

### Read Signal Data CSV and Import as Pandas DataFrame

In [2]:
# Read in data and set index
filepath = Path('../Resources/link_trading_signals.csv')
signals_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
signals_df = signals_df.set_index("Timestamp", drop=True)
signals_df.head()

,Close,RSI1,short,long,rsi_signal,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Timestamp,,,,,,,,,,,,
2020-06-11 02:00:00,0.000447,0.000000,0.0,1.0,1.0,10000.0,10000.0,4.4717,95.5283,100.0000,NaN,NaN
2020-06-11 03:00:00,0.000446,0.000000,0.0,1.0,1.0,10000.0,10000.0,8.9230,91.0668,99.9898,-0.000102,-0.000102
2020-06-11 04:00:00,0.000445,0.000000,0.0,1.0,1.0,10000.0,10000.0,13.3647,86.6119,99.9766,-0.000132,-0.000234
2020-06-11 05:00:00,0.000445,0.000000,0.0,1.0,1.0,10000.0,10000.0,17.8112,82.1591,99.9703,-0.000063,-0.000297
2020-06-11 06:00:00,0.000447,43.740253,0.0,0.0,0.0,0.0,0.0,17.8624,82.1591,100.0215,0.000512,0.000215


### Prepare Portfolio Evaluation Metrics DataFrame

In [3]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [4]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,-0.503687
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [5]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,-0.150829
Cumulative Returns,-0.503687
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [6]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,-0.150829
Cumulative Returns,-0.503687
Annual Volatility,0.434465
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [7]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,-0.150829
Cumulative Returns,-0.503687
Annual Volatility,0.434465
Sharpe Ratio,-0.347161
Sortino Ratio,NaN


In [8]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,-0.150829
Cumulative Returns,-0.503687
Annual Volatility,0.434465
Sharpe Ratio,-0.347161
Sortino Ratio,NaN


In [9]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,-0.150829
Cumulative Returns,-0.503687
Annual Volatility,0.434465
Sharpe Ratio,-0.347161
Sortino Ratio,-0.477734


### Prepare Trade Evaluation Metrics DataFrame

In [10]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Coin', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Coin,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Calculate and Append Trade Evaluation Metrics

In [11]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['rsi_signal'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['rsi_signal'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Holdings']
#         exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position']) + row['Portfolio Cash']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Coin': 'LINK',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Coin,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,LINK,2020-06-15 04:00:00,2020-06-16 07:00:00,10000.0,0.000410,0.000424,98.3568,97.5660,-0.7908
1,LINK,2020-06-15 04:00:00,2020-06-16 08:00:00,10000.0,0.000410,0.000428,98.3568,94.1600,-4.1968
2,LINK,2020-06-15 04:00:00,2020-06-16 09:00:00,10000.0,0.000410,0.000429,98.3568,89.9955,-8.3613
3,LINK,2020-06-15 04:00:00,2020-06-16 10:00:00,10000.0,0.000410,0.000427,98.3568,85.4640,-12.8928
4,LINK,2020-06-15 04:00:00,2020-06-17 07:00:00,10000.0,0.000410,0.000429,98.3568,81.4530,-16.9038
5,LINK,2020-06-15 04:00:00,2020-06-17 08:00:00,10000.0,0.000410,0.000429,98.3568,77.2002,-21.1566
6,LINK,2020-06-15 04:00:00,2020-06-17 09:00:00,10000.0,0.000410,0.000429,98.3568,72.8875,-25.4693
7,LINK,2020-06-15 04:00:00,2020-06-17 11:00:00,10000.0,0.000410,0.000429,98.3568,68.6608,-29.6960
8,LINK,2020-06-15 04:00:00,2020-06-17 12:00:00,10000.0,0.000410,0.000430,98.3568,64.4400,-33.9168
9,LINK,2020-06-15 04:00:00,2020-06-17 20:00:00,10000.0,0.000410,0.000441,98.3568,61.7288,-36.6280
